
<h1><center>Natural Language Inference</center></h1>

Мы продолжим знакомство с моделями на основе архитектуры Трансформер.
Мы рассмотрим классификатор на основе модели RoBERTa (модель на базе BERT), дообученный для решения задачи Natural Language Inference (NLI). В этой задаче нам требуется предсказать, если ли логическая связь между двумя предложениями: следует ли одного из другого.  Этот классификатор, предобученный на задаче NLI, позволит нам решать задачу классификации в Zero-shot постановке. В такой постановке модели НЕ требуется обучение на размеченных данных, поскольку любую задачу мы можем свести к задаче NLI –- мы рассмотрим, как именно в примерах ниже.

Мы загрузим заранее обученный классификатор и будем использовать его, чтообы классифицировать тексты на нужные нам классы.

Два важных отличия рассматриваемой сегодня модели от тех, с которыми мы работали на предыдущих занятиях:
- Подход zero-shot не требует обучения модели, нам следует лишь подать классификатору текст и названия нужных нам классов. Модель будет строить предсказание на основе своего внутреннего представления, обученного заранее.
- Данная модель многоязычна. Мы будем работать с текстами, написанными на разных языках.

## Классификация текстов на английском языке

Установим библиотеку transformers и загрузим модуль pipeline, в котором находится нужная нам модель.

In [1]:
#!pip install transformers

In [13]:
!pip install -q sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.7 MB/s eta 0:00:00


In [ ]:
#!pip install -qq accelerate

In [1]:
from transformers import pipeline

In [4]:
classifier = pipeline("zero-shot-classification")
# classifier = pipeline("zero-shot-classification", device=0) # to utilize GPU

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.



С помощью этой модели мы можем классифицировать тексты как принадлежащие или не принадлежащие одному из нужных нам классов.

По умолчанию предполагается, что текст принадлежит ровно к одному из объявленных классов, а модель возвращает вероятности каждого из классов, которые в сумме равны 1.

Посмотрим простой пример.

In [5]:
sequence = "Who are you voting for in 2020?"
candidate_labels = ["politics", "public health", "economics"]

classifier(sequence, candidate_labels)

{'sequence': 'Who are you voting for in 2020?',
 'labels': ['politics', 'economics', 'public health'],
 'scores': [0.9725187420845032, 0.014584163203835487, 0.012897036969661713]}

Модель также может решать задачу многоклассоовой классификации, определяя для каждого класса, принадлежит ли к нему текст или нет.
В этом случае модель возвращает вероятности каждого из классов независимо. Нам понадобиться добавить парамерт ```multi_class=True``` в вызов функции.

In [6]:
sequence = "Who are you voting for in 2020?"
candidate_labels = ["politics", "public health", "economics", "elections"]

classifier(sequence, candidate_labels, multi_class=True)

The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.


{'sequence': 'Who are you voting for in 2020?',
 'labels': ['politics', 'elections', 'public health', 'economics'],
 'scores': [0.9720695614814758,
  0.9676108360290527,
  0.032487425953149796,
  0.0061645060777664185]}

Мы можем решать не только задачу классификации по темам, но и задачу классификации по тональности: можно классифицировать отзывы на негативный и позитивный классы.

In [7]:
sequence = "I hated this movie. The acting sucked."
candidate_labels = ["positive", "negative"]

classifier(sequence, candidate_labels)

{'sequence': 'I hated this movie. The acting sucked.',
 'labels': ['negative', 'positive'],
 'scores': [0.9916267395019531, 0.00837323535233736]}


Модель, лежащая в основе данного классификатора, была обучена на задаче Natural Language Inference (NLI). Задача состояла в следующем: по двум входящим текстам требовалось определить, является ли один из них продолжением другого.

Такую модель можно использвать в задаче zero-shot классификации, если свести классификацию к задаче NLI. Для этого модели на вход подаются два текста:
- текст, который нужно классифицировать (*предпосылка*)
- текст шаблона, в который вставлено название нужного класса (*гипотеза*)


Если с точки зрения NLI-модели текст гипотезы продолжает текст предпосылки, то мы можем заключить, что классифицируемый текст относится к соответствующему классу.

По умолчанию заданные метки классов вставляются в шаблон `This example is {class_name}.`

Во многих случаях такой подход работает достаточно хорошо, но в некоторых задачах качество классификации можно улучшить, используя более подходящий шаблон.

In [8]:
sequences = [
    "I hated this movie. The acting sucked.",
    "This movie didn't quite live up to my high expectations, but overall I still really enjoyed it."
]
candidate_labels = ["positive", "negative"]

classifier(sequences, candidate_labels)

[{'sequence': 'I hated this movie. The acting sucked.',
  'labels': ['negative', 'positive'],
  'scores': [0.9916267395019531, 0.00837323535233736]},
 {'sequence': "This movie didn't quite live up to my high expectations, but overall I still really enjoyed it.",
  'labels': ['negative', 'positive'],
  'scores': [0.8148515820503235, 0.1851484328508377]}]

Второй пример выше несколько сложнее первого, поэтому модель предсказывает негативный класс менее уверенно.

Попробуем повысить уверенность классификации, используя более подходящий шаблон для данной задачи. Вместо шаблона по умолчанию `This example is {}.`, мы будем использовать `The sentiment of this review is {}.` (здесь `{}` будет заменено на название класса)

In [9]:
sequences = [
    "I hated this movie. The acting sucked.",
    "This movie didn't quite live up to my high expectations, but overall I still really enjoyed it."
]
candidate_labels = ["positive", "negative"]
hypothesis_template = "The sentiment of this review is {}."

classifier(sequences, candidate_labels, hypothesis_template=hypothesis_template)

[{'sequence': 'I hated this movie. The acting sucked.',
  'labels': ['negative', 'positive'],
  'scores': [0.9890092611312866, 0.010990714654326439]},
 {'sequence': "This movie didn't quite live up to my high expectations, but overall I still really enjoyed it.",
  'labels': ['positive', 'negative'],
  'scores': [0.9581226706504822, 0.041877321898937225]}]


Используя более точные и более подходящие к нашему контексту шаблоны, мы можем получить более точную классификацию.

## Многоязычная классификация

Данный классификатор можно использовать не только для английского языка, но и для ряда других. Для этого была обучена кросс-язычная модель на базе модели XLM RoBERTa.

Чтобы использовать эту модель, достаточно добавить дополнительный параметр ```model``` при объявлении классификатора.

In [2]:
classifier = pipeline("zero-shot-classification",
                       model="vicgalle/xlm-roberta-large-xnli-anli")



Модель позволяет использовать любую комбинацию языков. Можно, например, классифицировать русский текст с английскими названиями классов.

In [3]:
sequence = "За кого вы голосуете в 2020 году?" # translation: "Who are you voting for in 2020?"
candidate_labels = ["Europe", "public health", "politics"]

classifier(sequence, candidate_labels)

{'sequence': 'За кого вы голосуете в 2020 году?',
 'labels': ['politics', 'Europe', 'public health'],
 'scores': [0.9972042441368103, 0.0018147701630368829, 0.0009809574112296104]}

Теперь можно сделать то же самое, только поменять названия классов на французские.

In [4]:
sequence = "За кого вы голосуете в 2020 году?" # translation: "Who are you voting for in 2020?"
candidate_labels = ["Europe", "santé publique", "politique"]

classifier(sequence, candidate_labels)

{'sequence': 'За кого вы голосуете в 2020 году?',
 'labels': ['politique', 'Europe', 'santé publique'],
 'scores': [0.9980378150939941, 0.001338488538749516, 0.0006236950866878033]}


Как было отмечено выше, по умолчанию в модели используется шаблон на английском языке, `This text is {}.`
В случае, если мы работаем с другим языком, имеет смысл поменять данный шаблон на аналогичный, написанный на нужном нам языке.

In [5]:
sequence = "¿A quién vas a votar en 2020?"
candidate_labels = ["Europa", "salud pública", "política"]
hypothesis_template = "Este ejemplo es {}."

classifier(sequence, candidate_labels, hypothesis_template=hypothesis_template)

{'sequence': '¿A quién vas a votar en 2020?',
 'labels': ['política', 'Europa', 'salud pública'],
 'scores': [0.9983715415000916, 0.0012645989190787077, 0.00036386377178132534]}

Модель была дообучена на корпусе XNLI, в состав которого входят тексты на 15 языках: английский, арабский, болгарский, вьетнамский, греческий, испанский, немецкий, русский, суахили, тайский, турецкий, урду, французский и хинди.

Базовая модель RoBERTa, на основе которой была построена данная модель, была обучена на 100 языках, поэтому для каждого из этих языков эта модель будет также работать в некоторой степени.  



## LLMs Prompting ##

Современные большие языковые модели (Large Language Models, LLMs) можно использовать без дообучения, подбирая для них формулировки задач, также называемые промптами (prompts), аналогично тому, как мы задавали promt для модели RoBERTa. Выбирая различные промпты, можно решать не только задачу классификации, но и другие NLP задачи: извлечение сущностей (NER), генерацию текста, машинный перевод, суммаризацию и тд.

В зависимости от решаемой задачи, важно выбрать тип модели, который вы будете использовать. Все модели, построенные на архитектуре Трансформер, состоят либо только из энкодера (BERT, RoBERTa), либо только из декодера (GPT, LLaMa), либо из обеих частей (BART, T5).

Энкодер модели на выходе производят эмбеддинги слов и предложений, которые можно использовать для внешних задач - мы рассматривали эти примеры ранее в курсе.

Модели, которые состоят из декодера (или энкодера и декодера) - это авторегрессионные модели, обученные на задаче предсказания следующего токена, поэтому после обучения их можно использовать в режиме генерации текста, и, подбирая правильные промпты, решать задачи обработки текста.


In [2]:
from transformers import AutoTokenizer

###  Начнем с простой генерации текста: ###

Так можно запускать генерацию для моделей, состоящих только из декодера, например, модели gpt2:

In [3]:
model = pipeline('text-generation', model = 'gpt2')

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [10]:
prompt = "I woke up this morning feeling"

model(prompt, max_length = 30)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'I woke up this morning feeling this light pulsed, and wanted to get him in my arms and let him make this happen," says Jodi.'}]

Для использования энкодер-декодер модели нужен pipleine "text2text-generation":

In [12]:
model = pipeline("text2text-generation", model = 'google/flan-t5-base')

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [15]:
prompt = "I woke up this morning and"

model(prompt, max_length = 30)

[{'generated_text': 'I was so tired. I had to go to the bathroom.'}]

### Теперь перейдем к другим задачам!###

In [4]:
import torch

In [4]:
model = "tiiuae/falcon-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model)

In [5]:
pipe = pipeline(
     "text-generation",
     model=model,
     tokenizer=tokenizer,
     torch_dtype=torch.bfloat16,
     device_map="auto",
)

In [7]:
prompt = """
Classify the following text into neutral, negative or positive.
Text: I installed the app 3 days ago and it stopped working today. I paid 3$ for the features and I can't use them. The support was not helpfull at all.
Sentiment:
"""

sequences = pipe(
     prompt,
     max_new_tokens=10,
)

for seq in sequences:
  print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Result: 
Classify the following text into neutral, negative or positive. 
Text: I installed the app 3 days ago and it stopped working today. I paid 3$ for the features and I can't use them. The support was not helpfull at all.
Sentiment:
I have an Android and I use it with iOS


Бывает! Попробуем взять модель побольше.

Для использования больших моделей в сценарии ограниченных ресурсов (например, Google Colab) мы будем использовать библиотеку Accelerate. Если вы используете Google Colab, не забудьте первым шагом переключиться на рантайм побольше (например, T4 GPU).

Подробный туториал: https://huggingface.co/blog/accelerate-large-models

Принцип работы Accelarate следующий:

1. Инициализируюм пустую модель (без весов)
2. Далее мы будем использовать все доступные девайсы; слои модели разложим по разным девайсам
3. Загружаем часть весов в память
4. Эти веса добавляем в пустую модель
5. Переносим веса на девайс, где будут производиться вычисления
6. Повторяем шаги 3-5 для оставшихся весов.





In [5]:
!pip install -q accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.9 MB/s eta 0:00:00


In [7]:
from accelerate import init_empty_weights, infer_auto_device_map
from transformers import AutoConfig, AutoModelForCausalLM

In [10]:
model_name = "facebook/opt-13b"

config = AutoConfig.from_pretrained(model_name)
with init_empty_weights():
    model = AutoModelForCausalLM.from_config(config)

model.tie_weights()
device_map = infer_auto_device_map(model)

In [11]:
# Save the model weights
torch.save(model.state_dict(), 'model_weights.pth')

In [ ]:
device_map["model.decoder.layers.37"] = "disk"
model = AutoModelForCausalLM.from_pretrained(
    model_name, device_map=device_map, offload_folder="offload", offload_state_dict = True, torch_dtype=torch.float16
)

pytorch_model.bin:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

In [17]:
tokenizer = AutoTokenizer.from_pretrained(model)

pipe = pipeline(
    "text-generation",
    model = model,
    tokenizer = tokenizer
    )

Классификация:

In [18]:
prompt = """
Classify the following text into neutral, negative or positive.
Text: I installed the app 3 days ago and it stopped working today. I paid 3$ for the features and I can't use them. The support was not helpfull at all.
Sentiment:
"""

sequences = pipe(
     prompt,
     max_new_tokens=10,
)

for seq in sequences:
  print(f"Result: {seq['generated_text']}")

Result: negative


Named Entity Recognition:

In [25]:
prompt = """
Return a list of named entities in the following text.
Text: This morning I'm going to see St. Basil's Cathedral in Red Square.
Named entities:
 """

sequences = pipe(
     prompt,
     max_new_tokens=50,
 )

for seq in sequences:
     print(f"{seq['generated_text']}")

St. Basil's Cathedral


Модель нашла только одну из сущностей. Попробуем подход **few-shot learning:** сперва подадим в модель пример верно решенной задачи, а потом - наш тестовый пример:

In [31]:
prompt = """
Text: This morning I'm going to see the Kremlin in Red Square.
Named entities: Kremlin, Red Square
Text: This morning I'm going to see Berlin Cathedral in Museum Island.
Named entities:
 """

sequences = pipe(
     prompt,
     max_new_tokens=50,
 )

for seq in sequences:
     print(f"{seq['generated_text']}")

Berlin Cathedral, Museum Island


Гораздо лучше!

Машинный перевод:

In [34]:
prompt = """
Translate the English text to German.
Text: This morning I don't feel like doing anything.
Translation:
"""

sequences = pipe(
     prompt,
     max_new_tokens=50,
 )

for seq in sequences:
     print(f"{seq['generated_text']}")

Ich halte diesen Morgen nichts zu tun.


Суммаризация:

In [38]:
prompt = """
The Classical period falls between the Baroque and the Romantic periods. Classical music has a lighter, clearer texture than Baroque music, but a more varying use of musical form, which is, in simpler terms, the rhythm and organization of any given piece of music. It is mainly homophonic, using a clear melody line over a subordinate chordal accompaniment, but counterpoint was by no means forgotten, especially in liturgical vocal music and, later in the period, secular instrumental music.
Write a summary of the above text.
Summary:
"""

sequences = pipe(
     prompt,
     max_new_tokens=30,
     do_sample=True,
     top_k=10,
 )

for seq in sequences:
     print(f"{seq['generated_text']}")

The history of the Classical period consists of three different periods: the Renaissance, the Baroque, and the Romantic.


QA (Question Answering):

In [40]:
prompt = """
Answer the question using the context below.
Context: The Classical period falls between the Baroque and the Romantic periods. Classical music has a lighter, clearer texture than Baroque music, but a more varying use of musical form.
Question: Does Baroque music have clearer texture, than Classical one?
Answer:
"""

sequences = pipe(
     prompt,
     max_new_tokens=10,
     do_sample=True,
     top_k=10,
 )

for seq in sequences:
     print(f"Result: {seq['generated_text']}")

Result: no
